In [1]:
# Import functions and libraries
from __future__ import division
%pylab
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pyaudio
import Queue
import threading,time
import sys
from numpy import *
from scipy import *
import scipy.signal as signal
from rtlsdr import RtlSdr
from numpy.fft import *

import serial
%matplotlib inline


Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pyaudio
import wave

def play_audio( Q, p, fs , dev, ser="", keydelay=0):
    # play_audio plays audio with sampling rate = fs
    # Q - A queue object from which to play
    # p   - pyAudio object
    # fs  - sampling rate
    # dev - device number
    # ser - pyserial device to key the radio
    # keydelay - delay after keying the radio
    
    # Example:
    # fs = 44100
    # p = pyaudio.PyAudio() #instantiate PyAudio
    # Q = Queue.queue()
    # Q.put(data)
    # Q.put("EOT") # when function gets EOT it will quit
    # play_audio( Q, p, fs,1 ) # play audio
    # p.terminate() # terminate pyAudio
    
    # open output stream
    ostream = p.open(format=pyaudio.paFloat32, channels=1, rate=int(fs),output=True,output_device_index=dev)
    # play audio
    while (1):
        data = Q.get()
        if data=="EOT"  :
            break
        elif (data=="KEYOFF"  and ser!=""):
            ser.setDTR(0)
            #print("keyoff\n")
        elif (data=="KEYON" and ser!=""):
            ser.setDTR(1)  # key PTT
            #print("keyon\n")
            time.sleep(keydelay) # wait 200ms (default) to let the power amp to ramp up
            
        else:
            try:
                ostream.write( data.astype(np.float32).tostring() )
            except:
                print("Exception")
                break
            
def record_audio( queue, p, fs ,dev,chunk=1024):
    # record_audio records audio with sampling rate = fs
    # queue - output data queue
    # p     - pyAudio object
    # fs    - sampling rate
    # dev   - device number 
    # chunk - chunks of samples at a time default 1024
    #
    # Example:
    # fs = 44100
    # Q = Queue.queue()
    # p = pyaudio.PyAudio() #instantiate PyAudio
    # record_audio( Q, p, fs, 1) # 
    # p.terminate() # terminate pyAudio
    
   
    istream = p.open(format=pyaudio.paFloat32, channels=1, rate=int(fs),input=True,input_device_index=dev,frames_per_buffer=chunk)

    # record audio in chunks and append to frames
    frames = [];
    while (1):
        try:  # when the pyaudio object is distroyed stops
            data_str = istream.read(chunk) # read a chunk of data
        except:
            break
        data_flt = np.fromstring( data_str, 'float32' ) # convert string to float
        queue.put( data_flt ) # append to list


In [3]:
def printDevNumbers(p):
    N = p.get_device_count()
    for n in range(0,N):
        name = p.get_device_info_by_index(n).get('name')
        print n, name

In [4]:
p = pyaudio.PyAudio()
printDevNumbers(p)
p.terminate()

0 Built-in Microph
1 Built-in Output


In [5]:
# dusb_in = 2
# dusb_out = 2
din = 0
dout = 1

In [6]:
?RtlSdr

In [7]:
sample_rate = 240000
center_freq = 97.3 #443.610e6
gain = 36
N_samples = 8*256000 # approximately seconds

sdr0 = RtlSdr(device_index=0)
sdr1 = RtlSdr(device_index=1)
sdr2 = RtlSdr(device_index=2)
sdr0.sample_rate = sample_rate     # sampling rate  1000000
sdr0.center_freq = center_freq   # 162MhZ center frequency 162e6
sdr0.gain = gain 
sdr1.sample_rate = sample_rate     # sampling rate  1000000
sdr1.center_freq = center_freq   # 162MhZ center frequency 162e6
sdr1.gain = gain 
sdr2.sample_rate = sample_rate     # sampling rate  1000000
sdr2.center_freq = center_freq   # 162MhZ center frequency 162e6
sdr2.gain = gain 

y0 = sdr0.read_samples(N_samples)
sdr0.close()
y1 = sdr1.read_samples(N_samples)
sdr1.close()
y2 = sdr2.read_samples(N_samples)
sdr2.close()

In [8]:
## BELOW DOESN'T WORK

## Play sounds

fs = 44100

Qout = Queue.Queue()

# create a pyaudio object
p = pyaudio.PyAudio()

Qout = Queue.Queue()

Qout.put(y0)

play_audio(Qout, p, sample_rate, dout)

while not(Qout.empty()) :
    time.sleep(1)

Qout.put(y1)

play_audio(Qout, p, sample_rate, dout)

while not(Qout.empty()) :
    time.sleep(1)
    
Qout.put(y2)

while not(Qout.empty()) :
    time.sleep(1)
    
# kill the pyaudio device. The thread will have an exception and will quit. 
p.terminate()


IOError: [Errno Invalid sample rate] -9997